# Rag for H and M

### Put Your OPENROUTER_API_KEY here

In [ ]:
import os
os.environ["OPENROUTER_API_KEY"] = "paste_your_api_key_here"

In [ ]:
!pip3 install langchain langchain_openai

In [ ]:
import os
from langchain_openai import ChatOpenAI

try:
    model_name
except NameError:
    model_name="openai/gpt-4o-mini"

print("Model Name:", model_name)
print("Provider:", "OpenRouter AI")

llm=ChatOpenAI(model_name=model_name,
               openai_api_key=os.environ.get("OPENROUTER_API_KEY"),
               openai_api_base="https://openrouter.ai/api/v1")

In [ ]:
import textwrap
from langchain_core.prompts import ChatPromptTemplate

from IPython.display import display, clear_output, Markdown
from ipywidgets import widgets, Layout

conversation_output = widgets.Output()
messages = []

def run_chatbot(system_prompt, initial_message):
    global messages 
    messages = [ {'role':'system', 'content': system_prompt} ]
    conversation_output.clear_output()

    messages.append({'role': 'assistant', 'content': initial_message})

    text_input = widgets.Text(
        placeholder='Type your message here...',
        layout=widgets.Layout(width='50%')
    )
    submit_button = widgets.Button(description="Send")

    input_box = widgets.HBox([text_input, submit_button])
    display(conversation_output, input_box)

    def on_submit_click(b):
        message = text_input.value
        text_input.value = ''  # Clear the input field

        with conversation_output:
            display(Markdown(f"**User**: {message}"))
            messages.append({'role': 'user', 'content': message})
            response = get_completion_messages(messages)
            display(Markdown(f"**AI**: {response}"))
            messages.append({'role': 'assistant', 'content': response})

    submit_button.on_click(on_submit_click)

    # Display initial AI message
    with conversation_output:
        display(Markdown(f"**AI**: {initial_message}"))

def wrap_text(text, max_width=80):
    """
    Wraps the text to the specified max_width, preserving line breaks and formatting.
    """
    text = text.lstrip()
    lines = text.splitlines()  # Split the text into lines
    wrapped_lines = []
    for line in lines:
        if line.strip():  # Skip empty lines
            wrapped_line = textwrap.fill(line, max_width, initial_indent='', subsequent_indent='')
            wrapped_lines.extend(wrapped_line.splitlines())  # Preserve line breaks
        else:
            wrapped_lines.append('')  # Keep empty lines
    return '\n'.join(wrapped_lines)

def print_messages():
    for message in messages:
        role = message['role']
        content = message['content']
        
        if role == 'system':
            print("System:")
            print("-" * 40)
            print(content)
        elif role == 'user':
            print("User: ", end="")
            print(wrap_text(content))
        elif role == 'assistant':
            print("Assistant: ", end="")
            print(wrap_text(content))
        print()  # Add an extra newline for spacing

def print_prompt_and_response(prompt, response):
    print("Prompt: ")
    print(wrap_text(prompt))
    print("")
    print("Response: ")
    print(response)

def print_messages_and_response(messages, response):
    prompt = ChatPromptTemplate(messages=messages)
    print_prompt_and_response(prompt.format(), response)

def get_completion(prompt, temperature=0.0):
    response = llm.invoke(prompt, temperature=temperature)
    wrapped_response = wrap_text(response.content)
    return wrapped_response

def get_completion_messages(messages, temperature=0.0):
    response=llm.invoke(messages, temperature=temperature)
    wrapped_response = wrap_text(response.content)
    return wrapped_response

In [ ]:
import numpy as np
import pandas as pd 
from sentence_transformers import SentenceTransformer
import time
from io import StringIO
from io import BytesIO
from typing import List, Tuple
import requests

In [ ]:
class ProductSearcher:
    def __init__(self, embeddings_url: str = "https://github.com/calvinw/semantic-search/raw/refs/heads/main/product_embeddings.npz"):
        """Initialize the searcher with embeddings from GitHub URL."""
        print("Downloading embeddings file from GitHub...")
        try:
            response = requests.get(embeddings_url)
            response.raise_for_status()
            
            # Load the embeddings from the downloaded content
            print("Loading embeddings and model...")
            self.data = np.load(BytesIO(response.content), allow_pickle=True)
            self.embeddings = self.data['embeddings']
            self.product_names = self.data['product_names']
            self.embedding_strings = self.data['embedding_strings']
            self.product_codes = self.data['product_codes']
            self.article_ids_str = self.data['article_ids_str']
            
            # Load the model
            print("Loading sentence transformer model...")
            self.model = SentenceTransformer('all-MiniLM-L6-v2')
            print("Model loaded successfully!")
            
            print(f"Loaded {len(self.embeddings)} products")
            
        except Exception as e:
            print(f"Error in initialization: {e}")
            raise

    def get_product_name(self, idx):
        return self.product_names[idx]

    def get_embedding_string(self, idx):
        return self.embedding_strings[idx]
    
    def search(self, query: str, top_k: int = 4) -> List[Tuple[int, float]]:
        """Search for products using a text query."""
        query_embedding = self.model.encode([query])[0]
        
        similarities = np.dot(self.embeddings, query_embedding) / (
            np.linalg.norm(self.embeddings, axis=1) * np.linalg.norm(query_embedding)
        )
        
        top_idx = np.argsort(similarities)[::-1][:top_k]
        return [(idx, similarities[idx]) for idx in top_idx]
    
searcher = ProductSearcher()

## A Search and Results

In [ ]:
query="Socks for holiday"
hits=searcher.search(query, 3)

idx_first = hits[0][0]
idx_second = hits[1][0]
idx_third = hits[2][0]

score1 = hits[0][1]
score2 = hits[1][1]
score3 = hits[2][1]

match1 = searcher.get_embedding_string(idx_first)
match2 = searcher.get_embedding_string(idx_second)
match3 = searcher.get_embedding_string(idx_third)

match1 = str(score1) + "|" + match1
match2 = str(score2) + "|" + match2
match3 = str(score3) + "|" + match3

print(match1)

## Bundle the Matches and Send to LLM

Let’s set up a prompt to ask the LLM to discuss the hits and ask them if
they want more information about the items.

In [ ]:
template="""
Below are three matched items that were returned by vector search of a user
query, which is in the <query> </query> tags. The first number is the
similarity score of the match. 

Please formulate a response to the user's query that describes the 3 items and
asks the user if they 

<query>
{query}
</query>

<match>
{match1}
</match>

<match>
{match2}
</match>

<match>
{match3}
</match>
"""

prompt=template.format(match1=match1, match2=match2, match3=match3, query=query)
response = get_completion(prompt)

print("Prompt:")
print(prompt)
print("Response:")
print(response)